# <center>Laboratorium Analiza i bazy danych </center>

## <center>Tidy data i TIER protocol</center>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Load the Data

In [2]:
weather = pd.read_csv("../OriginalData/weather.txt", header=None)
days = (weather[0].map(lambda x: x[21:])
    .str.replace("OI", "  ")
    .str.replace("OS", "  ")
    .str.replace("SI", "  ")
    .str.replace("I", " ")
    .str.replace("S", " ")
    .str.replace("B", " ")
    .str.replace("D", " ")
    .map(str.lstrip)
    .str.split(r"\s+", expand=True)
    )[list(range(31))].rename(columns={i: f"d{i+1}" for i in range(31)})
days = days.replace("-9999", np.NaN)
for i in range(1,32):
    days[f"d{i}"] = days[f"d{i}"].astype(float)/10

weather_df = pd.DataFrame()
weather_df['id'] = weather[0].map(lambda x: x[:11])
weather_df['year'] = weather[0].map(lambda x: x[11:15]).astype(int)
weather_df['month'] = weather[0].map(lambda x: x[15:17]).astype(int)
weather_df['variable name'] = weather[0].map(lambda x: x[17:21]).str.lower()
for i in range(1,32):
    weather_df[f"d{i}"] = days[f"d{i}"]
weather_df

,id,year,month,variable name,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX000017004,1955,4,tmax,31.0,31.0,31.0,32.0,33.0,32.0,...,33.0,33.0,33.0,33.0,33.0,33.0,34.0,33.0,32.0,NaN
1,MX000017004,1955,4,tmin,15.0,15.0,16.0,15.0,16.0,16.0,...,17.0,17.0,17.0,18.0,19.0,19.0,17.0,18.0,16.0,NaN
2,MX000017004,1955,4,prcp,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,NaN
3,MX000017004,1955,5,tmax,31.0,31.0,31.0,30.0,30.0,30.0,...,33.0,34.0,35.0,33.0,31.0,31.0,32.0,31.0,30.0,29.0
4,MX000017004,1955,5,tmin,20.0,16.0,16.0,15.0,15.0,15.0,...,17.0,19.0,19.0,19.0,18.0,16.0,15.0,17.0,15.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,MX000017004,2011,3,tmin,NaN,NaN,NaN,NaN,14.8,NaN,...,NaN,NaN,NaN,NaN,16.2,NaN,NaN,NaN,NaN,17.0
1710,MX000017004,2011,3,prcp,0.0,NaN,0.0,0.0,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
1711,MX000017004,2011,4,tmax,NaN,35.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1712,MX000017004,2011,4,tmin,NaN,16.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tidying data

In [3]:
weather_in_2010 = weather_df[(weather_df["year"] == 2010)]
weather_in_2010 = weather_in_2010.reset_index(drop=True)

molten_weather = pd.melt(weather_in_2010, id_vars=["id", "year", "month", "variable name"], var_name="day")
molten_weather["day"] = molten_weather["day"].map(lambda x: x[1:]).astype(int)
molten_weather["date"] = pd.to_datetime(molten_weather[["year", "month", "day"]], errors="coerce")

molten_weather = molten_weather[["id", "date", "variable name", "value"]]
molten_weather = molten_weather[molten_weather["value"].notnull()]

molten_weather = molten_weather.sort_values(["id", "date", "variable name"])
molten_weather = molten_weather.reset_index(drop=True)
molten_weather

,id,date,variable name,value
0,MX000017004,2010-01-05,prcp,0.0
1,MX000017004,2010-01-07,prcp,0.0
2,MX000017004,2010-01-12,prcp,0.0
3,MX000017004,2010-01-14,prcp,0.0
4,MX000017004,2010-01-15,prcp,0.0
...,...,...,...,...
285,MX000017004,2010-12-24,prcp,0.0
286,MX000017004,2010-12-28,prcp,0.0
287,MX000017004,2010-12-29,prcp,0.0
288,MX000017004,2010-12-30,prcp,0.0


Tidy data

In [9]:
tidy_weather = molten_weather.set_index(["id", "date", "variable name"]).unstack()

tidy_weather.columns = tidy_weather.columns.droplevel(0)
tidy_weather.columns.name = None
tidy_weather = tidy_weather.reset_index()

temp = tidy_weather[["id","date","tmin","tmax"]]
temp = temp.dropna()
temp = temp.reset_index(drop=True)
temp.to_csv('../AnalysisData/temperature.csv', index=False)

prcp = tidy_weather[["id","date","prcp"]]
prcp = prcp.dropna()
prcp = prcp.reset_index(drop=True)
prcp.to_csv('../AnalysisData/precipitation.csv', index=False)
temp

,id,date,tmin,tmax
0,MX000017004,2010-01-30,14.5,27.8
1,MX000017004,2010-02-02,14.4,27.3
2,MX000017004,2010-02-03,14.4,24.1
3,MX000017004,2010-02-11,13.4,29.7
4,MX000017004,2010-02-23,10.7,29.9
5,MX000017004,2010-03-05,14.2,32.1
6,MX000017004,2010-03-10,16.8,34.5
7,MX000017004,2010-03-16,17.6,31.1
8,MX000017004,2010-04-27,16.7,36.3
9,MX000017004,2010-05-27,18.2,33.2


In [10]:
prcp

,id,date,prcp
0,MX000017004,2010-01-05,0.0
1,MX000017004,2010-01-07,0.0
2,MX000017004,2010-01-12,0.0
3,MX000017004,2010-01-14,0.0
4,MX000017004,2010-01-15,0.0
...,...,...,...
219,MX000017004,2010-12-24,0.0
220,MX000017004,2010-12-28,0.0
221,MX000017004,2010-12-29,0.0
222,MX000017004,2010-12-30,0.0
